# Лабораторная работа №5
## Выполнил студент группы ФИО ГРУППА

In [41]:
#Дополнительные модули, использованные при выполнение лабораторной
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

### Алгоритм Кнута-Морриса-Пратта

In [34]:
def KMP(text: str, pattern: str, ignore_сase=False) -> bool:
    if ignore_сase:
        text = text.lower()
        pattern = pattern.lower()
    t = len(text)
    p = len(pattern)
    if p > t:
        return False
    # массив содержащий максимальные длины суффиксов для каждого элемента в подстроке
    max_suffix = [0]*p
    j = 0
    for i in range(1, p-1):
        if pattern[j] == pattern[i]:
            max_suffix[i] = j+1
            j += 1
        else:
            if j == 0:
                max_suffix[i] = 0
            else:
                j = max_suffix[j-1]
    i, j = 0, 0 # i - итератор по тексту, j - итератор по подстроке
    while i < t:
        if text[i] == pattern[j]:
            i += 1
            j += 1
            if j == p:
                return True # подстрока найдена
        else:
            if j > 0:
                j = max_suffix[j-1]
            else:
                i += 1
    if i == t and j != p:
        return False    

In [38]:
pattern, options = input("Подстрока для поиска: "), bool(input("Ингорировать регистр (True/False): ").title())
with open("sources/1.txt", "r", encoding="utf-8") as f:
    data = f.read()
if KMP(data, pattern, options):
    print("Подстрока найдена")
else:
    print("Подстрока не найдена")

Подстрока для поиска: война 
Ингорировать регистр (True/False): true
Подстрока найдена


### Упрощенный алгоритм Бойера-Мура

In [67]:
def BM(text: str, pattern: str, ignore_сase=False) -> bool:
    # значение по умолчанию для смещения, если буква отсутствует в паттерне
    def default_value():
        return len(pattern) 
    
    if ignore_сase:
        text = text.lower()
        pattern = pattern.lower()
    t = len(text)
    p = len(pattern)
    if p > t:
        return False
    d = defaultdict(default_value) # словарь смещений
    #заполнение словаря смещений
    for i in range(p-2, -1, -1):
        if pattern[i] not in d:
            d[pattern[i]] = p-i-1
    if pattern[p-1] not in d:
        d[pattern[p-1]] = p
    i = p-1 # счетчик проверяемого символа в тексте
    while(i < t):
        k = 0 # указатель внутри текста k = [0, len(text)-1], охватывает предыдущие символы в тексте
        j = 0 # указатель внутри паттерна
        mismatch = False
        for j in range(p-1, -1, -1): 
            if text[i-k] != pattern[j]:
                if j == p-1:
                    offset = d[text[i]] # смещение, если не равен последний символ образа
                else:
                    offset = d[pattern[j]] # смещение, если не равен не последний символ образа
                i += offset # смещение счетчика строки
                mismatch = True # если несовпадение символа, то mismatch = True
                break
            k += 1
        if not mismatch:
            return True
    return False

In [71]:
pattern, options = input("Подстрока для поиска: "), bool(input("Ингорировать регистр (True/False): ").title())
with open("sources/1.txt", "r", encoding="utf-8") as f:
    data = f.read()
if BM(data, pattern, options):
    print("Подстрока найдена")
else:
    print("Подстрока не найдена")

Подстрока для поиска: string
Ингорировать регистр (True/False): 
Подстрока не найдена
